In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt

import wradlib
from osgeo import osr
import datetime as dt
import glob
from osgeo import gdal
from scipy.spatial import cKDTree
import wradlib as wrl
import matplotlib as mpl

#% matplotlib inline 

import datetime as dt

In [2]:
def corcorcor(A,B):
    from scipy import stats, linspace

        
    try:
        mask = ~np.isnan(A) & ~np.isnan(B)
        diffi = A[mask]-B[mask]
        bias = np.nansum(diffi)/len(diffi)
        slope, intercept, r_value, p_value, std_err = stats.linregress(A[mask], B[mask])
        line = slope * A +intercept
        corr, bias = np.round(r_value,2),  np.round(bias,2)
    except:
        corr, bias = np.nan, np.nan
    
    return corr, bias

In [3]:
def skill_score(estimate, reference, th):
    """
    estimate
    
    reference
    
    threshold
    """
    # SkillScore berechnung
    #contigency tables Lit: Tang et al. 2015
    from scipy import stats, linspace
    
    val1 = estimate.copy()
    val2 = reference.copy()

    H_pos = np.array((val1 > th) & (val2 > th))
    M_pos = np.array((val1 <= th) & (val2 > th))
    F_pos = np.array((val1 > th) & (val2 <= th))
    C_pos = np.array((val1 <= th) & (val2 <= th))

    H = float(len(H_pos[H_pos==True]))
    M = float(len(M_pos[M_pos==True]))
    F = float(len(F_pos[F_pos==True]))
    C = float(len(C_pos[C_pos==True]))

    #maske = ~np.isnan(val1) & ~np.isnan(val2)
    #N = float(len(maske[maske==True]))
    N = np.nansum([H, M, F, C])
    
    # Rates
    try:
        E = 1./N * (((H + M)*(H + F)) + ((C + M) * (C + F)))
    except:
        E = np.nan

    try:
        POD = H/(H + M)
    except:
        POD = np.nan   

    try:    
        FAR = F/(H + F)
    except:
        FAR = np.nan

    try:
        BID = (H +F)/(H + M)
    except:
        BID = np.nan

    try:    
        HSS = (H + C - E) / (N - E)
    except:
        HSS = np.nan
    try:
        HR = (H+C)/N
    except:
        HR = np.nan
    #print (H, M, F, C, N, E)
    #print (HSS, FAR, BID, POD)


    #
    val1[val1<th]=np.nan
    val2[val2<th]=np.nan
    maske2 = ~np.isnan(val1) & ~np.isnan(val2)
    diffi = val1[maske2]-val2[maske2]
    summi = np.nansum(val2[maske2])
    #bias
    bias = np.nansum(diffi)/len(diffi)
    rmse = np.sqrt(np.nansum(((diffi)**2.0)/len(diffi)))

    slope, intercept, r_value, p_value, std_err = stats.linregress(val1[maske2], val2[maske2])
    
    PE = 100. * np.nansum(diffi)/summi
    
    #print(r_value, bias, rmse)
    result = {'H': H,
              'M': M,
              'F': F,
              'C': C,
              'N': N,
              'POD': POD,
              'FAR': FAR,
              'BID': BID,
              'HSS': HSS,
              'bias': bias,
              'RMSE': rmse,
              'corr':r_value,
             'HR':HR,
             'PE':PE}

    #for key, value in result.items():
    #    print(key + ':', value)

    return result

In [4]:
def plotplot(ax, A, B, tit):
    ff=20
    from matplotlib.colors import LogNorm



    #pl.hist2d(ry1[(rhmin1>bbh1+bbw1/2)&(dprh1>bbh1+bbw1/2)],
    #           dpr1[(rhmin1>bbh1+bbw1/2)&(dprh1>bbh1+bbw1/2)],
    #          bins=100, vmin=1, vmax=100, norm=LogNorm())

    #fig, ax = plt.subplots(figsize=(10,10))
    #ax = pl.subplot()
    xbins = 10**np.arange(-3, 3, 0.1)
    ybins = 10**np.arange(-3, 3, 0.1)

    counts, _, _ = np.histogram2d(A,
                                  B,
                                  bins=(xbins, ybins))

    a = np.ma.masked_where(counts.T == 0., counts.T)
    pl.pcolormesh(xbins, ybins, a, cmap='jet', vmin=0)
    print str(corcorcor(A,B))
    
    cb = plt.colorbar(shrink=0.5)
    cb.set_label('number of samples', fontsize=ff)
    cb.ax.tick_params(labelsize=20)

    ax.set_xscale('log')
    ax.set_yscale('log')

    pl.title(tit,fontsize=ff)
    pl.ylabel('DPR Rainrate in mm/h',fontsize=ff)
    pl.xlabel('Radolan Rainrate in mm/h',fontsize=ff)
    
    pl.xticks(fontsize=ff)
    pl.yticks(fontsize=ff)
    pl.plot([0,1000], [0,1000], color='black', label='r: '+ str(corcorcor(A,B)[0])+'\n bias: '+ str(corcorcor(A,B)[1]))
    pl.legend(loc='lower right', markerscale=0)


CPU times: user 0 ns, sys: 13.1 ms, total: 13.1 ms
Wall time: 27.5 ms


Es gibt 39 Overpasses


In [5]:
%%time
pfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado*")[1:40]
print ('Es gibt ' + str(len(pfad)) + ' Overpasses')

para_dict = {'sr_x': [],
             'sr_y': [],
             'gr_h_max': [],
             'gr_h_mean': [],
             'gr_h_min': [],
             'gr_h_std': [],
             ##!memory!##'gr_pp': [],
             'gr_pp_ipoli': [],
             ##### 'gr_time': np.array([]),
             ##!memory!##'gr_xy': [],
             'sr_bbh': [],
             'sr_bbw': [],
             'sr_cfb': [],
             'sr_ele': [],
             'sr_lst': [],
             'sr_pav': [],
             'sr_pes': [],
             'sr_phase': [],
             'sr_pns': [],
             'sr_sth': [],
             ##### 'sr_time': np.array([]),
             'sr_times': [],
             'sr_type': []
}   


time_delta = np.array([])

a=0

for dict_name in para_dict.keys():
    # DR als liste
    print(dict_name)
    
    if 'gr' in dict_name:
        gr_name = 'GR'
    else:
        gr_name = 'SR'
    
    for p in pfad:

        DR = h5py.File(p, 'r')
        # for über liste
        a+=1
        # Listen append x.append(x)
        para_dict[dict_name].append(DR[gr_name][dict_name][:])
        
        if dict_name == 'sr_times':
            
        
        # ! Colse hdf5 file ! 
        DR.close()
        del(DR)
        
        
    ac = np.concatenate(para_dict[dict_name],axis=0)
    print (ac.shape)

    #np.save('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/'+dict_name+'.npy',ac) 
    para_dict.pop(dict_name,None)

'%%time\npfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado*")\n\nprint (\'Es gibt \' + str(len(pfad)) + \' Overpasses\')\n\npara_dict = {\'sr_x\': [],\n             \'sr_y\': [],\n             \'gr_h_max\': [],\n             \'gr_h_mean\': [],\n             \'gr_h_min\': [],\n             \'gr_h_std\': [],\n             ##!memory!##\'gr_pp\': [],\n             \'gr_pp_ipoli\': [],\n             ##### \'gr_time\': np.array([]),\n             ##!memory!##\'gr_xy\': [],\n             \'sr_bbh\': [],\n             \'sr_bbw\': [],\n             \'sr_cfb\': [],\n             \'sr_ele\': [],\n             \'sr_lst\': [],\n             \'sr_pav\': [],\n             \'sr_pes\': [],\n             \'sr_phase\': [],\n             \'sr_pns\': [],\n             \'sr_sth\': [],\n             ##### \'sr_time\': np.array([]),\n             \'sr_times\': [],\n             \'sr_type\': []\n}   \n\na=0\n\nfor dict_name in para_dict.keys():\n    # DR als liste\n    print(dict_na

In [5]:
"""%%time
pfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado*")

print ('Es gibt ' + str(len(pfad)) + ' Overpasses')

para_dict = {'sr_x': [],
             'sr_y': [],
             'gr_h_max': [],
             'gr_h_mean': [],
             'gr_h_min': [],
             'gr_h_std': [],
             ##!memory!##'gr_pp': [],
             'gr_pp_ipoli': [],
             ##### 'gr_time': np.array([]),
             ##!memory!##'gr_xy': [],
             'sr_bbh': [],
             'sr_bbw': [],
             'sr_cfb': [],
             'sr_ele': [],
             'sr_lst': [],
             'sr_pav': [],
             'sr_pes': [],
             'sr_phase': [],
             'sr_pns': [],
             'sr_sth': [],
             ##### 'sr_time': np.array([]),
             'sr_times': [],
             'sr_type': []
}   

a=0

for dict_name in para_dict.keys():
    # DR als liste
    print(dict_name)
    
    if 'gr' in dict_name:
        gr_name = 'GR'
    else:
        gr_name = 'SR'
    
    for p in pfad:

        DR = h5py.File(p, 'r')
        # for über liste
        a+=1
        # Listen append x.append(x)
        para_dict[dict_name].append(DR[gr_name][dict_name][:])
        
        # ! Colse hdf5 file ! 
        DR.close()
        del(DR)
        
        
    ac = np.concatenate(para_dict[dict_name],axis=0)
    print (ac.shape)

    np.save('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/'+dict_name+'.npy',ac) 
    para_dict.pop(dict_name,None)"""

'%%time\npfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado*")\n\nprint (\'Es gibt \' + str(len(pfad)) + \' Overpasses\')\n\npara_dict = {\'sr_x\': [],\n             \'sr_y\': [],\n             \'gr_h_max\': [],\n             \'gr_h_mean\': [],\n             \'gr_h_min\': [],\n             \'gr_h_std\': [],\n             ##!memory!##\'gr_pp\': [],\n             \'gr_pp_ipoli\': [],\n             ##### \'gr_time\': np.array([]),\n             ##!memory!##\'gr_xy\': [],\n             \'sr_bbh\': [],\n             \'sr_bbw\': [],\n             \'sr_cfb\': [],\n             \'sr_ele\': [],\n             \'sr_lst\': [],\n             \'sr_pav\': [],\n             \'sr_pes\': [],\n             \'sr_phase\': [],\n             \'sr_pns\': [],\n             \'sr_sth\': [],\n             ##### \'sr_time\': np.array([]),\n             \'sr_times\': [],\n             \'sr_type\': []\n}   \n\na=0\n\nfor dict_name in para_dict.keys():\n    # DR als liste\n    print(dict_na

In [6]:
xl = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_x.npy')
yl = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_y.npy')

sr_pns = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_pns.npy')
sr_pav = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_pav.npy')
gr_pns = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/gr_pp_ipoli.npy')

gr_hmin = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/gr_h_min.npy')
gr_hmax = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/gr_h_max.npy')
sr_ele = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_ele.npy')

sr_typ = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_type.npy')
sr_pha = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_phase.npy')
sr_cfb = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_cfb.npy')

sr_bbh = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_bbh.npy')
sr_bbw = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_bbw.npy')
sr_sth = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_sth.npy')

sr_lst = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_lst.npy')

In [ ]:
skill_score(gr_pns, sr_pav, 0.5)

In [ ]:
th_set = np.arange(0,1,0.01)
#h_set = np.array([0,1,0.1])


In [ ]:
res1 = res2 = np.array([])

for i in th_set:
    #print i
    res1 = np.append(res1, skill_score(sr_pns[sr_pha==2], gr_pns[sr_pha==2], th=i))
    #print skill_score(sr_pns, gr_pns, th=i)['corr']
    res2 = np.append(res2, skill_score(sr_pav[sr_pha==2], gr_pns[sr_pha==2], th=i))
    #print skill_score(sr_pav, gr_pns, th=i)['corr']

In [ ]:
res2[1]

In [ ]:
pl.figure(figsize=(15,10))
for i in range(len(res1)):
    #print th_set[i], res1[i][wort] 
    pl.subplot(3,4,1)
    wort = 'HR'
    #pl.yscale('log')
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.title('pns: green, pav:blue')
    pl.grid()
    
    pl.subplot(3,4,2)
    wort = 'corr'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,3)
    wort = 'bias'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,4)
    wort = 'RMSE'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,5)
    wort = 'POD'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,6)
    wort = 'BID'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,7)
    wort = 'FAR'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,8)
    wort = 'HSS'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,9)
    wort = 'H'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,10)
    wort = 'F'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,11)
    wort = 'M'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()
    
    pl.subplot(3,4,12)
    wort = 'PE'
    pl.scatter(th_set[i], res1[i][wort], color='green')
    #pl.scatter(th_set[i], res2[i][wort], color = 'blue')
    pl.axhline(0)
    pl.axvline(0.2, color='red')
    pl.axvline(0.5, color='black')
    pl.ylabel(wort + ' - sr-gr')
    pl.xlabel('Threshold in mm/h')
    pl.grid()

pl.tight_layout()

In [7]:
#Umwandeln von typ
"""
a: Main rain type. (a=1,2,3),
b: 0,
c: 0,
d: V rain type,
e: H rain type,
f: BB in outter sath detected,
g: Shallow rain,
h: Small size cell

Major rain type = typePrecip/10000000
    = 1    stratiform
    = 2    convective
    = 3    other

b = 0: BB not detected,
    1: BB detected.

g = 0: No shallow rain,
    1: Shallow isolated,
    3: Shallow non-isolated.

h = 0: No small size cell,
    1: Single cell,
    2: Small size cell consisting of two adjacent pixels.
"""
typ = sr_typ.copy()

typ = typ//10000000

# Umwandeln von Phase.. Achtung es ist schon Umgewandelt
"""
phaseNearSurface < 100 Temperature(C)=phaseNearSurface-100
phaseNearSurface > 200 Temperature(C)=phaseNearSurface-200
phaseNearSurface = 100 Top of the bright band
phaseNearSurface = 200 Bottom of the bright band
phaseNearSurface = 125 is used for the range bins between
                    the top and peak of bright band
phaseNearSurface = 175 is used for the range bins between
                    the peak and bottom of bright band
                    
Integer values of phaseNearSurface/100 =
0 - solid
1 - mixed phase
2 - liquid
255 - Missing
"""
pha = sr_pha.copy()
pha = pha/100

print(np.unique(pha))
# Umwandeln von clutterfree bottom
#cfb = sr_cfb.copy()
#cfb[cfb<0]=np.nan
#cfb = 125.*(176. - cfb)


[0 2]


In [ ]:
pl.figure(figsize=(18,14))
xbins = 10**np.arange(-3, 3, 0.1)

pl.subplot(4,3,1)
pl.xscale('log')
pl.hist(sr_pns[sr_pns>0], bins=xbins)
pl.xlabel('Rainrate (mm/h)', fontsize=20)
pl.title('PNS', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,2)
pl.xscale('log')
pl.hist(sr_pav[sr_pav>0], bins=xbins)
pl.xlabel('Rainrate (mm/h)', fontsize=20)
pl.title('PAV', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,3)
pl.xscale('log')
pl.hist(gr_pns[gr_pns>0], bins=xbins)
pl.xlabel('Rainrate (mm/h)', fontsize=20)
pl.title('GRP', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,4)
pl.hist(sr_bbh[sr_bbh>0]/1000, bins=50)
pl.xlabel('Height (km)', fontsize=20)
pl.title('BBH', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,5)
pl.hist(sr_bbw[sr_bbw>0]/1000, bins=50)
pl.xlabel('Width (km)', fontsize=20)
pl.title('BBW', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,6)
pl.hist(sr_sth[sr_sth>=0]/1000, bins=50)
pl.xlabel('Height (km)', fontsize=20)
pl.title('STH', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,7)
pl.hist(sr_lst[(sr_lst>=0) & (sr_lst!=-9999)], bins=30)
pl.xlim(0,400)
pl.xlabel('Ocean ----  Land  ---- Coast ---- InlandWater')
pl.title('LST', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,8)
pl.hist(typ[typ>0])
pl.xlabel('Strati ---- Convectiv ---- Other')
pl.title('TYP', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,9)
pl.yscale('log')
pl.hist(sr_pha[sr_pha>=0], bins=5)
pl.xlabel('Solid ---- Mixed  ---- Liquid')
pl.title('PHA', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,10)
srh = sr_cfb - sr_ele
pl.hist(srh[srh>=0]/1000, bins=30)
pl.xlim(0,6)
pl.xlabel('Height (km)', fontsize=20)
pl.title('CFB', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,12)
pl.hist(gr_hmin[gr_hmin>=0]/1000., bins=30)
pl.xlim(0,6)
pl.xlabel('Height (km)', fontsize=20)
pl.title('GRH', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.subplot(4,3,11)
pl.hist(sr_ele[sr_ele>0]/1000, bins=30)
pl.xlim(0,6)
pl.xlabel('Height (km)', fontsize=20)
pl.title('ELE', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

pl.tight_layout()
pl.show()

In [ ]:
# new

# Hoehen unterschied zwischen Sr un GR der 3 Jahre
-------------------------------------------------------------

In [ ]:
srh = sr_cfb-sr_ele
grh = gr_hmin 
ff = 20
def get_miub_cmap():
    import matplotlib.colors as col
    startcolor = 'white'  # a dark olive
    color1 = '#8ec7ff'#'cyan'    # a bright yellow
    color2 = 'dodgerblue'
    color3 = 'lime'
    color4 = 'yellow'
    color5 = 'darkorange'
    color6 = 'red'
    color7 = 'purple'
    #color6 = 'grey'
    endcolor = 'darkmagenta'    # medium dark red
    colors = [startcolor, color1, color2, color3, color4, color5, color6, endcolor]
    return col.LinearSegmentedColormap.from_list('miub1',colors)


mh = ~np.isnan(grh) & ~np.isnan(srh)

pl.hist2d(srh[mh], grh[mh], bins=200, cmap=get_miub_cmap() )
pl.xlabel("SR measuring height in m", fontsize=ff)
pl.ylabel("GR measuring height in m", fontsize=ff)
cb = pl.colorbar(shrink=0.5)
cb.set_label('number of samples', fontsize=ff)
cb.ax.tick_params(labelsize=20)
pl.xlim(0,4000)
pl.ylim(0,4000)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)

pl.plot([0,6000],[0,6000], linestyle="solid", color="black")
pl.tight_layout()
pl.show()

In [ ]:
pl.hist(grh[mh], bins=200, histtype='step', normed=False,label='GR', color='blue')
pl.hist(srh[mh], bins=200, histtype='step', normed=False, label='SR', color='green')
pl.legend()
pl.xlabel("Measuring height in m", fontsize=ff)
pl.grid()
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.show()

In [ ]:
srh = sr_cfb-sr_ele
grh = gr_hmin 

hdiff = grh - srh

ff = 20
def get_miub_cmap():
    import matplotlib.colors as col
    startcolor = 'white'  # a dark olive
    color1 = '#8ec7ff'#'cyan'    # a bright yellow
    color2 = 'dodgerblue'
    color3 = 'lime'
    color4 = 'yellow'
    color5 = 'darkorange'
    color6 = 'red'
    color7 = 'purple'
    #color6 = 'grey'
    endcolor = 'darkmagenta'    # medium dark red
    colors = [startcolor, color1, color2, color3, color4, color5, color6, endcolor]
    return col.LinearSegmentedColormap.from_list('miub1',colors)


mh = ~np.isnan(hdiff) & ~np.isnan(yl)

pl.figure(figsize=(12,12))
pl.hist2d(hdiff[mh], yl[mh], bins=200, cmap=get_miub_cmap() )
pl.xlabel("GR - SR measuring height in m", fontsize=ff)
pl.ylabel("y in km", fontsize=ff)
cb = pl.colorbar(shrink=0.5)
cb.set_label('number of samples', fontsize=ff)
cb.ax.tick_params(labelsize=20)
pl.xlim(-4000,4000)

pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.axvline(0, linestyle="solid", color="black")
pl.axvline(-1000, linestyle="-.", color="black")
pl.axvline(1000, linestyle="-.", color="black")

pl.tight_layout()
pl.show()

In [ ]:
srh = sr_cfb-sr_ele
grh = gr_hmin 

hdiff = grh - srh

ff = 20
def get_miub_cmap():
    import matplotlib.colors as col
    startcolor = 'white'  # a dark olive
    color1 = '#8ec7ff'#'cyan'    # a bright yellow
    color2 = 'dodgerblue'
    color3 = 'lime'
    color4 = 'yellow'
    color5 = 'darkorange'
    color6 = 'red'
    color7 = 'purple'
    #color6 = 'grey'
    endcolor = 'darkmagenta'    # medium dark red
    colors = [startcolor, color1, color2, color3, color4, color5, color6, endcolor]
    return col.LinearSegmentedColormap.from_list('miub1',colors)


mh = ~np.isnan(hdiff) & ~np.isnan(xl)

pl.figure(figsize=(12,12))
pl.hist2d(xl[mh], hdiff[mh], bins=200, cmap=get_miub_cmap() )
pl.ylabel("GR - SR measuring height in m", fontsize=ff)
pl.xlabel("x in km", fontsize=ff)
cb = pl.colorbar(shrink=0.5)
cb.set_label('number of samples', fontsize=ff)
cb.ax.tick_params(labelsize=20)
#pl.xlim(-4000,4000)

pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.axhline(0, linestyle="solid", color="black")
pl.axhline(-1000, linestyle="-.", color="black")
pl.axhline(1000, linestyle="-.", color="black")

pl.tight_layout()
pl.show()

# Höhenabhängigkeitsbeweis?
-------------------------------------------

In [8]:
ff=20

gr, sr = gr_pns.copy(), sr_pns.copy()
hmin, cfb, ele = gr_hmin.copy(), sr_cfb.copy(), sr_ele.copy()

gr[gr==0.]=np.nan
sr[sr==0.]=np.nan

#stratiform 1
gr[typ!=1.]=np.nan
sr[typ!=1.]=np.nan

#liquid 2
gr[pha!=2.]=np.nan
sr[pha!=2.]=np.nan

#pos = (gr_hmin - (sr_cfb - sr_ele)) < 500.
pl.figure(figsize=(15,5))
ax1 = pl.subplot(1,2,1, aspect='equal')
#pl.xscale('log')
#pl.yscale('log')
#pl.xlim(1e-3,1000)
#pl.ylim(1e-3,1000)
plotplot(ax1, gr, sr, 'All Measurements')
#pl.plot([0,1000], [0,1000], color='black')
skill_score(gr, sr, 0.5)

hdiff = 100.

ax2 = pl.subplot(1,2,2, aspect='equal')
#pl.xscale('log')
#pl.yscale('log')
#pl.xlim(1e-3,1000)
#pl.ylim(1e-3,1000)
plotplot(ax2, gr[abs((hmin - (cfb - ele))) < hdiff],
         sr[abs((hmin - (cfb - ele))) < hdiff], 'Only Measurements with diff \n in height: '+str(hdiff)+' m')
#pl.plot([0,1000], [0,1000], color='black')
skill_score(gr[abs((hmin - (cfb - ele))) < hdiff], sr[abs((hmin - (cfb - ele))) < hdiff], 0.5)

pl.tight_layout()
pl.show()

/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/numpy/lib/function_base.py:991: RuntimeWarning: invalid value encountered in greater_equal
  not_smaller_than_edge = (sample[:, i] >= edges[i][-1])


(0.62, 0.3)


/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in greater
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in less_equal
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in greater
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in greater
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in less_equal
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in less_equal
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:64: RuntimeWarning: invalid value encoun

(0.71, 0.44)


/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in less


In [ ]:
hdiff = 100.

for hdiff in np.arange(100,2000,100):
    print hdiff
    SS = skill_score(gr[abs((hmin - (cfb - ele))) < hdiff], sr[abs((hmin - (cfb - ele))) < hdiff], 0.5)
    plt.scatter(hdiff, SS['corr'], color='blue')
    plt.scatter(hdiff, SS['POD'], color='red')
    plt.scatter(hdiff, SS['HR'], color='green')
    plt.scatter(hdiff, SS['HSS'], color='black')
    plt.scatter(hdiff, SS['FAR'], color='grey')
    
plt.show()


In [ ]:
hdiff = 100.

for hdiff in np.arange(100,2000,100):
    print hdiff
    SS = skill_score(gr[(abs((hmin - (cfb - ele))) < hdiff) & (abs((hmin - (cfb - ele))) > hdiff-100.)],
                     sr[(abs((hmin - (cfb - ele))) < hdiff) & (abs((hmin - (cfb - ele))) > hdiff-100.)],
                     0.5)
    plt.scatter(hdiff, SS['N'], color='blue')
    plt.scatter(hdiff, SS['H'], color='red')
    plt.scatter(hdiff, SS['F'], color='green')
    plt.scatter(hdiff, SS['M'], color='black')
    plt.scatter(hdiff, SS['C'], color='grey')
    

plt.xlabel('hdiff in m')    
plt.show()


In [30]:
step = 125.
corr, bias, RMSE = np.array([]), np.array([]), np.array([])
N, H, F, M, C = np.array([]), np.array([]), np.array([]), np.array([]), np.array([])
POD, HSS, HR, BID, PE, FAR = np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])


_mean, _std = np.array([]), np.array([])

for hdiff in np.arange(step,3000,step):
    print hdiff
    SS = skill_score(gr[(abs((hmin - (cfb - ele))) < hdiff) & (abs((hmin - (cfb - ele))) > hdiff-step)],
                     sr[(abs((hmin - (cfb - ele))) < hdiff) & (abs((hmin - (cfb - ele))) > hdiff-step)],
                     0.5) 
    
    corr = np.append(corr, SS['corr'])
    bias = np.append(bias, SS['bias'])
    RMSE = np.append(RMSE, SS['RMSE'])
    POD= np.append(POD, SS['POD'])
    HSS = np.append(HSS, SS['HSS'])
    HR = np.append(HR, SS['HR'])
    BID = np.append(BID, SS['BID'])
    PE = np.append(PE, SS['PE'])
    FAR = np.append(FAR, SS['FAR'])
    N = np.append(N, SS['N'])
    H = np.append(H, SS['H'])
    F = np.append(F, SS['F'])
    M = np.append(M, SS['M'])
    C = np.append(C, SS['C'])

    
"""    plt.subplot(3,1,2)
    plt.scatter(hdiff, SS['N'],  marker='o', s=40)
    plt.scatter(hdiff, SS['H'],  marker='^', s=40)
    plt.scatter(hdiff, SS['F'],  marker='h', s=40)
    plt.scatter(hdiff, SS['M'],  marker='*', s=40)
    plt.scatter(hdiff, SS['C'],  marker='+', s=40)
    
    plt.legend()
    plt.grid()    
    plt.xlabel('hdiff in m')
    plt.ylabel('o:N, ^:H, h:F, *:M, +:C ') 
    
    plt.subplot(3,1,3)
    plt.scatter(hdiff, SS['POD'],  marker='o', s=40)
    plt.scatter(hdiff, SS['FAR'],  marker='^', s=40)
    plt.scatter(hdiff, SS['BID'],  marker='h', s=40)
    plt.scatter(hdiff, SS['HSS'],  marker='*', s=40)
    plt.scatter(hdiff, SS['PE']/100,  marker='+', s=40)
    plt.scatter(hdiff, SS['HR'],  marker='d', s=40)
    plt.grid()    
    plt.xlabel('hdiff in m')
    plt.ylabel('o:POD, ^:FAR, h:BID, *:HSS, +:PR , d:HR') 
    
    plt.subplot(3,1,1)
    plt.scatter(hdiff, SS['bias'], c=SS['corr'], vmin=0.3, vmax=0.7, marker='o', s=50)
    plt.scatter(hdiff, SS['RMSE'], c=SS['corr'], vmin=0.3, vmax=0.7, marker='^', s=50)
    plt.axhline(0)
    plt.grid()    
    plt.xlabel('hdiff in m')
    plt.ylabel('bias o ; RMSE ^ ') 

plt.colorbar()
plt.tight_layout()
plt.show()
"""

125.0


/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in less
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in greater
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in less
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in greater
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in greater
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in less_equal
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in gre

250.0
375.0
500.0
625.0
750.0
875.0
1000.0
1125.0
1250.0
1375.0
1500.0
1625.0
1750.0
1875.0
2000.0
2125.0
2250.0
2375.0
2500.0
2625.0
2750.0
2875.0


"    plt.subplot(3,1,2)\n    plt.scatter(hdiff, SS['N'],  marker='o', s=40)\n    plt.scatter(hdiff, SS['H'],  marker='^', s=40)\n    plt.scatter(hdiff, SS['F'],  marker='h', s=40)\n    plt.scatter(hdiff, SS['M'],  marker='*', s=40)\n    plt.scatter(hdiff, SS['C'],  marker='+', s=40)\n    \n    plt.legend()\n    plt.grid()    \n    plt.xlabel('hdiff in m')\n    plt.ylabel('o:N, ^:H, h:F, *:M, +:C ') \n    \n    plt.subplot(3,1,3)\n    plt.scatter(hdiff, SS['POD'],  marker='o', s=40)\n    plt.scatter(hdiff, SS['FAR'],  marker='^', s=40)\n    plt.scatter(hdiff, SS['BID'],  marker='h', s=40)\n    plt.scatter(hdiff, SS['HSS'],  marker='*', s=40)\n    plt.scatter(hdiff, SS['PE']/100,  marker='+', s=40)\n    plt.scatter(hdiff, SS['HR'],  marker='d', s=40)\n    plt.grid()    \n    plt.xlabel('hdiff in m')\n    plt.ylabel('o:POD, ^:FAR, h:BID, *:HSS, +:PR , d:HR') \n    \n    plt.subplot(3,1,1)\n    plt.scatter(hdiff, SS['bias'], c=SS['corr'], vmin=0.3, vmax=0.7, marker='o', s=50)\n    plt.scat

In [31]:
print np.count_nonzero(~np.isnan(sr- gr))
print np.nansum(N)

293906
293393.0


In [ ]:
hdiff = np.arange(step,3000,step)
ff=20
plt.figure(figsize=(20,14))
plt.subplot(3,1,2,aspect='auto')
plt.plot(hdiff, N,  marker='o', label='N')
plt.plot(hdiff, H,  marker='^', label='H')
plt.plot(hdiff, F,  marker='h', label='F')
plt.plot(hdiff, M,  marker='*', label='M')
plt.plot(hdiff, C,  marker='+', label='C')
plt.xlim(0, 4000)
plt.legend(loc='upper right',fontsize=ff)
plt.grid()    
plt.xticks(fontsize=ff)
plt.yticks(fontsize=ff)

# -------------------------------
plt.subplot(3,1,3,aspect='auto')
plt.plot(hdiff, corr,  marker='s', label='Corr', lw=3)
plt.plot(hdiff, POD,  marker='o', label='POD')
plt.plot(hdiff, FAR,  marker='^', label='FAR')
plt.plot(hdiff, BID,  marker='h', label='BID')
plt.plot(hdiff, HSS,  marker='*', label='HSS')
plt.plot(hdiff, PE/100,  marker='D', label='PE')
plt.plot(hdiff, HR,  marker='d', label='HR')
plt.axhline(0, color='grey', lw=4, alpha = 0.5)

plt.legend(loc='upper right',fontsize=ff)
plt.grid()    
plt.xlabel('hdiff in m',fontsize=ff)
plt.xlim(0, 4000)
plt.xticks(fontsize=ff)
plt.yticks(fontsize=ff)

# ------------------------------
plt.subplot(3,1,1,aspect='auto')
plt.plot(hdiff, bias, marker='o', color='blue', label='bias')
plt.plot(hdiff, RMSE, marker='^', color='black', label='RMSE')
plt.axhline(0, color='grey', lw=4)
plt.grid()    
plt.legend(loc='upper right',fontsize=ff)
plt.xticks(fontsize=ff)
plt.yticks(fontsize=ff)
plt.title('DPR NS vs. RY -  Liquid Stratiform 2014-2018, TH=0.5 mm/h, N_all:' + str(np.nansum(N)), fontsize=ff)


plt.xlim(0, 4000)

#plt.colorbar()
#plt.tight_layout()
plt.show()

In [ ]:
%%time

#pl.figure(figsize=(10,10))
#pl.scatter(para1, para2, c=para3, edgecolors='none', s=10)
#pl.xlim(-1,60)
#pl.ylim(-1,60)
#pl.colorbar()
#pl.plot([0,60],[0,60], linestyle="solid", color="black")
#pl.grid()

In [ ]:
dpr = sr_pns.copy() # para2 = GPM 
dpr2 = sr_pav.copy() # para2 = GPM 
ry = gr_pns.copy() # para1 = RY
xx = xl.copy()
yy = yl.copy()
#pha = np.array(sr_pha.copy(),dtype=float)
typ1 = typ.copy()
#dpr_h = ele_offest - sr_ele
#dpr_h = (178. - sr_cfb.copy())*125
#ry_h = gr_hmin.copy()

ff=20
#yy[(ry == 0) & (dpr > 0)] = np.nan
#xx[(ry == 0) & (dpr > 0)] = np.nan
#"""
th = 0.

#ry[ry <= th] = np.nan
#dpr[dpr <= th] = np.nan
#dpr2[dpr2 <= th] = np.nan

#typ[(ry <= th) & (dpr <= th) & (dpr2 <= th) ] = np.nan
#pha[(ry <= th) & (dpr <= th) & (dpr2 <= th) ] = np.nan

#mask = ~np.isnan(ry) & ~np.isnan(dpr) & ~np.isnan(dpr2) & ~np.isnan(typ) & ~np.isnan(pha)


#"""
#dpr_h[(ry <= 0) & (dpr <= 0) & (dpr2 <= 0)] = np.nan
#ry_h[(ry <= 0) & (dpr <= 0) & (dpr2 <= 0)] = np.nan

#ry[abs(ry_h-dpr_h)<500] = np.nan
#dpr[abs(ry_h-dpr_h)<500] = np.nan

#sth[(ry <= 0) & (dpr <= 0) & (dpr2 <= 0)] = np.nan





In [ ]:
np.unique(typ)

In [ ]:
pl.hist(typ[typ!=-1], bins=3,histtype='step', align='mid')
pl.xlim(0,4)


In [ ]:
%%time
ff=20
pl.figure(figsize=(10,6))
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[mask],dpr2[mask], c=typ[mask], edgecolors='none', cmap='jet')
print(corcorcor(ry[mask],dpr2[mask]))

cb = pl.colorbar()
cb.set_label('GR - SR height in m', fontsize=ff)
cb.ax.tick_params(labelsize=20)

pl.grid()

pl.ylabel('DPR Rainrate in mm/h',fontsize=ff)
pl.xlabel('Radolan Rainrate in mm/h',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.plot([0,1000], [0,1000], color='black')
#pl.savefig("/automount/ftp/velibor/test1.png")

In [ ]:
typ1= typ.astype(int)/10000000
print('___Corr______Err______BIAS____')

print(corcorcor(ry[pha==0.],dpr2[pha==0.]))
print(corcorcor(ry[pha==2.],dpr2[pha==2.]))

print('___________________')
print(corcorcor(ry[typ1==1.],dpr2[typ1==1.]))
print(corcorcor(ry[typ1==2.],dpr2[typ1==2.]))
print(corcorcor(ry[typ1==3.],dpr2[typ1==3.]))

print('___Min__________Max______')

print(np.nanmin(ry[ry>0]), np.nanmax(ry))
print(np.nanmin(dpr[dpr>0]), np.nanmax(dpr))
print(np.nanmin(dpr2[dpr2>0]), np.nanmax(dpr2))



In [ ]:
np.unique(sr_pha) # Keine mixed phase ? 

In [ ]:
%%time

ff=20
pl.figure(figsize=(15,15))
pl.subplot(3,2,1, aspect='equal')
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[(pha==0.) & (typ1==1.)],dpr[(pha==0.) & (typ1==1.)],  edgecolors='none', color='green', 
          label=str(corcorcor(ry[(pha==0.) & (typ1==1.)],dpr[(pha==0.) & (typ1==1.)])))
pl.plot([0,1000], [0,1000], color='black')
pl.ylabel('DPR RR in mm/h \n Stratiform',fontsize=ff)
pl.title('Solid',fontsize=ff, color='green')
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.legend()
pl.plot([0,1000], [0,1000], color='black')

pl.subplot(3,2,2, aspect='equal')
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[(pha==2.) & (typ1==1.)],dpr[(pha==2.) & (typ1==1.)],  edgecolors='none',
          label=str(corcorcor(ry[(pha==2.) & (typ1==1.)],dpr[(pha==2.) & (typ1==1.)])))
pl.plot([0,1000], [0,1000], color='black')
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.title('Liquid',fontsize=ff, color='blue')
pl.legend()
pl.plot([0,1000], [0,1000], color='black')

pl.subplot(3,2,3, aspect='equal')
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[(pha==0.) & (typ1==2.)],dpr[(pha==0.) & (typ1==2.)],  edgecolors='none', color='green',
          label=str(corcorcor(ry[(pha==0.) & (typ1==2.)],dpr[(pha==0.) & (typ1==2.)])))
pl.plot([0,1000], [0,1000], color='black')
pl.ylabel('DPR RR in mm/h \n Convective',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.legend()
pl.plot([0,1000], [0,1000], color='black')

pl.subplot(3,2,4, aspect='equal')
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[(pha==2.) & (typ1==2.)],dpr[(pha==2.) & (typ1==2.)],  edgecolors='none',
          label=str(corcorcor(ry[(pha==2.) & (typ1==2.)],dpr[(pha==2.) & (typ1==2.)])))
pl.plot([0,1000], [0,1000], color='black')
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.legend()
pl.plot([0,1000], [0,1000], color='black')

pl.subplot(3,2,5, aspect='equal')
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[(pha==0.) & (typ1==3.)],dpr[(pha==0.) & (typ1==3.)],  edgecolors='none', color='green',
          label=str(corcorcor(ry[(pha==0.) & (typ1==3.)],dpr[(pha==0.) & (typ1==3.)])))
pl.plot([0,1000], [0,1000], color='black')
pl.xlabel('Radolan RR in mm/h',fontsize=ff)
pl.ylabel('DPR RR in mm/h \n Other',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.legend()
pl.plot([0,1000], [0,1000], color='black')

pl.subplot(3,2,6, aspect='equal')
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[(pha==2.) & (typ1==3.)],dpr[(pha==2.) & (typ1==3.)],  edgecolors='none', 
          label=str(corcorcor(ry[(pha==2.) & (typ1==3.)],dpr[(pha==2.) & (typ1==3.)])))
pl.plot([0,1000], [0,1000], color='black')
pl.xlabel('Radolan RR in mm/h',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.legend()
pl.plot([0,1000], [0,1000], color='black')


pl.tight_layout()
#pl.savefig("/automount/ftp/velibor/test1.png")

In [ ]:
%%time

srh = sr_cfb-sr_ele
grh = gr_hmin 

hdiff = abs(grh - srh)

ff=20
pl.figure(figsize=(20,20))
ax1 = pl.subplot(3,2,1, aspect='equal')
#pl.xscale('log')
#pl.yscale('log')
#pl.xlim(1e-3,1000)
#pl.ylim(1e-3,1000)
plotplot(ax1, ry[(pha==0.) & (typ1==1.) & (hdiff<=100)],dpr[(pha==0.) & (typ1==1.)& (hdiff<=100)], 'Solid, Stratiform')
#pl.plot([0,1000], [0,1000], color='black')
#pl.ylabel('DPR RR in mm/h \n Stratiform',fontsize=ff)
#pl.title('Solid',fontsize=ff, color='green')
#pl.xticks(fontsize=ff)
#pl.yticks(fontsize=ff)
#pl.legend()
#pl.plot([0,1000], [0,1000], color='black')

ax2 = pl.subplot(3,2,2, aspect='equal')
#pl.xscale('log')
#pl.yscale('log')
#pl.xlim(1e-3,1000)
#pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
plotplot(ax2,ry[(pha==2.) & (typ1==1.)& (hdiff<=100)],dpr[(pha==2.) & (typ1==1.)& (hdiff<=100)],'Liquid, Stratiform')
#pl.plot([0,1000], [0,1000], color='black')
#pl.xticks(fontsize=ff)
#pl.yticks(fontsize=ff)
#pl.title('Liquid',fontsize=ff, color='blue')
#pl.legend()
#pl.plot([0,1000], [0,1000], color='black')

ax3 = pl.subplot(3,2,3, aspect='equal')
#pl.xscale('log')
#pl.yscale('log')
#pl.xlim(1e-3,1000)
#pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
plotplot(ax3, ry[(pha==0.) & (typ1==2.)& (hdiff<=100)],dpr[(pha==0.) & (typ1==2.)& (hdiff<=100)], 'Solid, Convective')
#pl.plot([0,1000], [0,1000], color='black')
#pl.ylabel('DPR RR in mm/h \n Convective',fontsize=ff)
#pl.xticks(fontsize=ff)
#pl.yticks(fontsize=ff)
#pl.legend()
#pl.plot([0,1000], [0,1000], color='black')

ax4 = pl.subplot(3,2,4, aspect='equal')
#pl.xscale('log')
#pl.yscale('log')
#pl.xlim(1e-3,1000)
#pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
plotplot(ax4,ry[(pha==2.) & (typ1==2.)& (hdiff<=100)],dpr[(pha==2.) & (typ1==2.)& (hdiff<=100)], 'Liquid Convective')
#pl.plot([0,1000], [0,1000], color='black')
#pl.xticks(fontsize=ff)
#pl.yticks(fontsize=ff)
#pl.legend()
#pl.plot([0,1000], [0,1000], color='black')

ax5 = pl.subplot(3,2,5, aspect='equal')

#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
plotplot(ax5, ry[(pha==0.) & (typ1==3.)& (hdiff<=100)],dpr[(pha==0.) & (typ1==3.)& (hdiff<=100)], 'Solid, Other')


ax6 = pl.subplot(3,2,6, aspect='equal')

#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
plotplot(ax6,ry[(pha==2.) & (typ1==3.)& (hdiff<=100)],dpr[(pha==2.) & (typ1==3.)& (hdiff<=100)],  'Liquid, Other')



pl.tight_layout()
pl.savefig("/automount/ftp/velibor/with_hdiff.png")
pl.show()

In [ ]:
pl.show()

In [ ]:
pl.figure(figsize=(20,20))
ax1 = pl.subplot(1,1,1, aspect='equal')
plotplot(ax1,ry[(pha==0.) & (typ1==1.)],dpr[(pha==0.) & (typ1==1.)], 
          str(corcorcor(ry[(pha==0.) & (typ1==1.)],dpr[(pha==0.) & (typ1==1.)])))

# Test GR und SR Messhoehen in abhängigkeit zum BB
--------------------------------------------

In [ ]:
dpr1 = sr_pns.copy() # para2 = GPM 
ry1 = gr_pns.copy() # para1 = RY

rhmax = gr_hmax.copy()
rhmin1 = gr_hmin.copy()
dprh1 = sr_cfb.copy()

bbh1 = sr_bbh.copy()
bbw1 = sr_bbw.copy()

bbh1[bbh1<=0]=np.nan
#bbh[bbh==0]=np.nan
dprh1[dprh1 < 0] = np.nan

In [ ]:
dprh1 = 125.*(176. - dprh1)

In [ ]:
dprh1[typ!=1]= np.nan # 1 =strati ; 2 =conv 

In [ ]:
dprh1[sr_pha!=2]= np.nan # 0=solid, 2=liquid

In [ ]:
np.unique(typ), np.unique(sr_pha) 

In [ ]:
# for hist2d only values without nan
bbh = bbh1[~np.isnan(bbh1) & ~np.isnan(bbw1) & ~np.isnan(rhmin1) & ~np.isnan(dprh1) & ~np.isnan(ry1) & ~np.isnan(dpr1)]
bbw = bbw1[~np.isnan(bbh1) & ~np.isnan(bbw1) & ~np.isnan(rhmin1) & ~np.isnan(dprh1) & ~np.isnan(ry1) & ~np.isnan(dpr1)]
dprh = dprh1[~np.isnan(bbh1) & ~np.isnan(bbw1) & ~np.isnan(rhmin1) & ~np.isnan(dprh1) & ~np.isnan(ry1) & ~np.isnan(dpr1)]
rhmin = rhmin1[~np.isnan(bbh1) & ~np.isnan(bbw1) & ~np.isnan(rhmin1) & ~np.isnan(dprh1) & ~np.isnan(ry1) & ~np.isnan(dpr1)]
ry = ry1[~np.isnan(bbh1) & ~np.isnan(bbw1) & ~np.isnan(rhmin1) & ~np.isnan(dprh1) & ~np.isnan(ry1) & ~np.isnan(dpr1)]
dpr = dpr1[~np.isnan(bbh1) & ~np.isnan(bbw1) & ~np.isnan(rhmin1) & ~np.isnan(dprh1) & ~np.isnan(ry1) & ~np.isnan(dpr1)]

### Wie oft misst RADOLAN über unter oder im BB
-------------------------

In [ ]:
pl.figure(figsize=(20,20))
ax1 = pl.subplot(3,3,1, aspect='equal')
plotplot(ax1,ry[(dprh<bbh+bbw/2)&(dprh>bbh-bbw/2)&(rhmin<bbh+bbw/2)&(rhmin>bbh-bbw/2)],
         dpr[(dprh<bbh+bbw/2)&(dprh>bbh-bbw/2)&(rhmin<bbh+bbw/2)&(rhmin>bbh-bbw/2)],
         'SR and GR in BB')

ax2 = pl.subplot(3,3,2, aspect='equal')
plotplot(ax2, ry[(dprh<bbh+bbw/2)&(dprh>bbh-bbw/2)&(rhmin>bbh+bbw/2)],
           dpr[(dprh<bbh+bbw/2)&(dprh>bbh-bbw/2)&(rhmin>bbh+bbw/2)], 'GR over & SR in BB')

ax3 = pl.subplot(3,3,8, aspect='equal')
plotplot(ax3, ry[(rhmin<bbh+bbw/2)&(rhmin>bbh-bbw/2)&(dprh>bbh+bbw/2)],
           dpr[(rhmin<bbh+bbw/2)&(rhmin>bbh-bbw/2)&(dprh>bbh+bbw/2)], 'GR in & SR over BB')

ax4 = pl.subplot(3,3,4, aspect='equal')
plotplot(ax4,ry[(dprh<bbh+bbw/2)&(dprh>bbh-bbw/2)&(rhmin<bbh-bbw/2)],
           dpr[(dprh<bbh+bbw/2)&(dprh>bbh-bbw/2)&(rhmin<bbh-bbw/2)], 'GR under & SR in BB')

ax5 = pl.subplot(3,3,5, aspect='equal')
plotplot(ax5, ry[(rhmin<bbh+bbw/2)&(rhmin>bbh-bbw/2)&(dprh<bbh-bbw/2)],
           dpr[(rhmin<bbh+bbw/2)&(rhmin>bbh-bbw/2)&(dprh<bbh-bbw/2)], 'GR in & SR under BB')

ax6 = pl.subplot(3,3,6, aspect='equal')
plotplot(ax6, ry[(rhmin>bbh+bbw/2)&(dprh<bbh-bbw/2)],
           dpr[(rhmin>bbh+bbw/2)&(dprh<bbh-bbw/2)], 'GR over & SR under BB')

ax7 = pl.subplot(3,3,7, aspect='equal')
plotplot(ax7, ry[(rhmin<bbh-bbw/2)&(dprh>bbh+bbw/2)],
               dpr[(rhmin<bbh-bbw/2)&(dprh>bbh+bbw/2)], 'GR under & SR over BB' )

ax8 = pl.subplot(3,3,3, aspect='equal')
plotplot(ax8, ry[(rhmin<bbh-bbw/2)&(dprh<bbh-bbw/2)],
           dpr[(rhmin<bbh-bbw/2)&(dprh<bbh-bbw/2)],'GR under & SR under BB')

ax9 = pl.subplot(3,3,9, aspect='equal')
plotplot(ax9, ry[(rhmin>bbh+bbw/2)&(dprh>bbh+bbw/2)],
           dpr[(rhmin>bbh+bbw/2)&(dprh>bbh+bbw/2)], 'GR over & SR over BB')

pl.tight_layout()

In [ ]:
#pl.yscale('log')
ff=20
pl.figure(figsize=(8,6))
pl.hist(rhmin[(rhmin<bbh+bbw/2)&(rhmin>bbh-bbw/2)], bins=30, histtype='step', label='in BB')
pl.hist(rhmin[(rhmin>bbh+bbw/2)], bins=30, histtype='step', label='over BB')
pl.hist(rhmin[(rhmin<bbh-bbw/2)], bins=30, histtype='step', label='under BB')
pl.legend(loc='upper right', fontsize=20)
pl.xlabel('RADOLAN measured height in m', fontsize=20)
pl.ylabel('number', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

In [ ]:
#pl.yscale('log')
ff=20
pl.figure(figsize=(8,6))
pl.hist(dprh[(dprh<bbh+bbw/2)&(dprh>bbh-bbw/2)], bins=21, histtype='step', label='in BB')
pl.hist(dprh[(dprh>bbh+bbw/2)], bins=21, histtype='step', label='over BB')
pl.hist(dprh[(dprh<bbh-bbw/2)], bins=21, histtype='step', label='under BB')
pl.legend(loc='upper right', fontsize=20)
pl.xlabel('DPR measured height in m', fontsize=20)
pl.ylabel('number', fontsize=20)
pl.xticks(fontsize=20)
pl.yticks(fontsize=20)

In [ ]:
ff=20

In [ ]:
def plotplot(ax, A, B, tit):
    ff=20
    from matplotlib.colors import LogNorm



    #pl.hist2d(ry1[(rhmin1>bbh1+bbw1/2)&(dprh1>bbh1+bbw1/2)],
    #           dpr1[(rhmin1>bbh1+bbw1/2)&(dprh1>bbh1+bbw1/2)],
    #          bins=100, vmin=1, vmax=100, norm=LogNorm())

    #fig, ax = plt.subplots(figsize=(10,10))
    #ax = pl.subplot()
    xbins = 10**np.arange(-3, 3, 0.1)
    ybins = 10**np.arange(-3, 3, 0.1)

    counts, _, _ = np.histogram2d(A,
                                  B,
                                  bins=(xbins, ybins))

    a = np.ma.masked_where(counts.T == 0., counts.T)
    pl.pcolormesh(xbins, ybins, a, cmap='jet', vmin=0)
    print str(corcorcor(A,B))
    
    cb = plt.colorbar(shrink=0.5)
    cb.set_label('number of samples', fontsize=ff)
    cb.ax.tick_params(labelsize=20)

    ax.set_xscale('log')
    ax.set_yscale('log')

    pl.title(tit,fontsize=ff)
    pl.ylabel('DPR Rainrate in mm/h',fontsize=ff)
    pl.xlabel('Radolan Rainrate in mm/h',fontsize=ff)
    
    pl.xticks(fontsize=ff)
    pl.yticks(fontsize=ff)
    pl.plot([0,1000], [0,1000], color='black', label='r: '+ str(corcorcor(A,B)[0])+'\n bias: '+ str(corcorcor(A,B)[1]))
    pl.legend(loc='lower right', markerscale=0)


In [ ]:
np.unique(dpr1[(rhmin1>bbh1+bbw1/2)&(dprh1>bbh1+bbw1/2)])


In [ ]:
len(np.array([])) +  len(np.array([0]))

In [ ]:
pl.xscale('log')
pl.yscale('log')

pl.xlim(th,1000)
pl.ylim(th,1000)
pl.scatter(ry[typ1==1.],dpr[typ1==1.])
pl.plot([0,1000], [0,1000], color='black')
corcorcor(ry[typ1==1.],dpr[typ1==1.])


In [ ]:
pl.xscale('log')
pl.yscale('log')
pl.xlim(th,1000)
pl.ylim(th,1000)
pl.scatter(ry[typ1==2.],dpr[typ1==2.])
pl.plot([0,1000], [0,1000], color='black')
corcorcor(ry[typ1==2.],dpr[typ1==2.])


In [ ]:
pl.xscale('log')
pl.yscale('log')
pl.xlim(th,1000)
pl.ylim(th,1000)
pl.scatter(ry[typ1==3.],dpr[typ1==3.])
pl.plot([0,1000], [0,1000], color='black')
corcorcor(ry[typ1==3.],dpr[typ1==3.])


In [ ]:
pl.xscale('log')
pl.yscale('log')
pl.xlim(th,1000)
pl.ylim(th,1000)
pl.scatter(ry[pha==2.],dpr[pha==2.])
pl.plot([0,1000], [0,1000], color='black')
corcorcor(ry[pha==2.],dpr[pha==2.])

In [ ]:
pl.xscale('log')
pl.yscale('log')
pl.xlim(th,1000)
pl.ylim(th,1000)
pl.scatter(ry[pha==0.],dpr[pha==0.])
pl.plot([0,1000], [0,1000], color='black')
corcorcor(ry[pha==0.],dpr[pha==0.])

In [ ]:
np.unique(typ1)

In [ ]:
mask = ~np.isnan(A) & ~np.isnan(B)
diffi = A[mask]-B[mask]
bias = np.nansum(diffi)/len(diffi)
slope, intercept, r_value, p_value, std_err = stats.linregress(A[mask], B[mask])


In [ ]:
%%time
ff=20
pl.figure(figsize=(10,6))
pl.xscale('log')
pl.yscale('log')
pl.xlim(1e-3,1000)
pl.ylim(1e-3,1000)
#pl.scatter(ry[mask],dpr[mask], c=sth[mask], edgecolors='none')
pl.scatter(ry[mask],dpr[mask], c=ry_h[mask]-dpr_h[mask], edgecolors='none', cmap='jet', vmin=-2000, vmax=2000)
print(corcorcor(ry[mask],dpr[mask]))
cb = pl.colorbar()
cb.set_label('storm height in m', fontsize=ff)
cb.ax.tick_params(labelsize=20)

pl.grid()

pl.ylabel('DPR Rainrate in mm/h',fontsize=ff)
pl.xlabel('Radolan Rainrate in mm/h',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.plot([0,1000], [0,1000], color='black')
#pl.savefig("/automount/ftp/velibor/test2.png")

In [ ]:
pl.figure(figsize=(10,6))
pl.hist(ry[mask],bins=np.logspace(np.log10(0.1), np.log10(100),1250),
        normed=True, cumulative=True, histtype='step', label='RY')
pl.hist(dpr[mask],bins=np.logspace(np.log10(0.1), np.log10(100),1250),
        normed=True, cumulative=True, histtype='step', label='PNS')
pl.hist(dpr2[mask],bins=np.logspace(np.log10(0.1), np.log10(100),1250),
        normed=True, cumulative=True, histtype='step', label='PAV')
pl.legend(fontsize=20, loc='lower right')
pl.xscale('log', nonposy='clip')
pl.grid()
pl.xlabel('RR (mm/h)', fontsize=20)
pl.ylabel('cdf', fontsize=20)
pl.xticks(fontsize=20);pl.yticks(fontsize=20)

In [ ]:
pl.figure(figsize=(10,6))
pl.hist(ry[mask],bins=np.logspace(np.log10(0.001), np.log10(100),250),
        normed=True, histtype='step', label='RY')
pl.hist(dpr[mask],bins=np.logspace(np.log10(0.001), np.log10(100),250),
        normed=True, histtype='step', label='PNS')
pl.hist(dpr2[mask],bins=np.logspace(np.log10(0.001), np.log10(100),250),
        normed=True, histtype='step', label='PAV')
pl.legend(fontsize=20, loc='lower right')
pl.xscale('log', nonposy='clip')
pl.grid()
pl.xlabel('RR (mm/h)', fontsize=20)
pl.ylabel('pdf', fontsize=20)
pl.xticks(fontsize=20);pl.yticks(fontsize=20)

In [ ]:
A, B = ry.copy(), dpr.copy()
C = sth.copy()

from pcc import get_my_cmap2



fig = pl.figure()
ax = pl.gca()
ax.scatter(A[mask],B[mask], c=C[mask], cmap=get_my_cmap2())
cb = pl.colorbar()
cb.set_label('sth', fontsize=ff)
#pl.plot([0,1000], [0,1000], lw=2, color='black')
ax.set_xscale('log')
ax.set_yscale('log')
pl.ylabel('DPR Rainrate in mm/h',fontsize=ff)
pl.xlabel('Radolan Rainrate in mm/h',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
pl.xlim(0,1000)
pl.ylim(0,1000)
#plt.title('Corr: ' + str(round(corr,3)) + r'$\pm$'+  str(round(eror,3))+
#            '\nBias: '+ str(round(bias,3)), fontsize=20)

pl.grid()
pl.show()
print corcorcor(A,B)

In [ ]:
A, B = ry.copy(), dpr.copy()
from pcc import get_my_cmap2

xbins = 10**np.arange(-1, 2.08, 0.08)
ybins = 10**np.arange(-1, 2.08, 0.08)

counts, _, _ = np.histogram2d(A[mask],B[mask], bins=(xbins, ybins))

fig, ax = pl.subplots(figsize=(10,10))
pl.pcolormesh(xbins, ybins, counts.T, cmap=get_my_cmap2(),vmin=0.1)
cb = pl.colorbar()
cb.set_label('number of samples', fontsize=ff)
pl.plot([0,1000], [0,1000], lw=2, color='black')
ax.set_xscale('log')
ax.set_yscale('log')
pl.ylabel('DPR Rainrate in mm/h',fontsize=ff)
pl.xlabel('Radolan Rainrate in mm/h',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)

#plt.title('Corr: ' + str(round(corr,3)) + r'$\pm$'+  str(round(eror,3))+
#            '\nBias: '+ str(round(bias,3)), fontsize=20)

pl.grid()
pl.show()
print corcorcor(A,B)

In [ ]:
A, B = ry.copy(), dpr2.copy()
from pcc import get_my_cmap2

xbins = 10**np.arange(-1, 2.08, 0.08)
ybins = 10**np.arange(-1, 2.08, 0.08)

counts, _, _ = np.histogram2d(A[mask],B[mask], bins=(xbins, ybins))

fig, ax = pl.subplots(figsize=(10,10))
pl.pcolormesh(xbins, ybins, counts.T, cmap=get_my_cmap2(),vmin=0.1)
cb = pl.colorbar()
cb.set_label('number of samples', fontsize=ff)
pl.plot([0,1000], [0,1000], lw=2, color='black')
ax.set_xscale('log')
ax.set_yscale('log')
pl.ylabel('DPR Rainrate in mm/h',fontsize=ff)
pl.xlabel('Radolan Rainrate in mm/h',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)

#plt.title('Corr: ' + str(round(corr,3)) + r'$\pm$'+  str(round(eror,3))+
#            '\nBias: '+ str(round(bias,3)), fontsize=20)

pl.grid()
pl.show()
print corcorcor(A,B)

### Overpass statistics

In [ ]:
def get_radar_locations():

    radars = {}
    radar = {}
    radar['name'] = 'ASR Dresden'
    radar['wmo'] = 10487
    radar['lon'] = 13.76347
    radar['lat'] = 51.12404
    radar['alt'] = 261
    radars['ASD'] = radar
    
    radar = {}
    #### Kordinaten un hoehe checken! Neu
    # Hoehe üNN 2m
    # Turm 55 m
    #radar['name'] = 'Borkum'
    #radar['wmo'] = 10103
    #radar['lon'] = 6.748292
    #radar['lat'] = 53.564011
    #radar['alt'] = 57.
    #radars['ASB'] = radar
    
    #Fake Borkum/Emden
    radar['name'] = 'Emden'
    radar['wmo'] = 10103
    radar['lon'] = 7.02377
    radar['lat'] = 53.33872
    radar['alt'] = 58.
    radars['ASB'] = radar

    radar = {}
    radar['name'] = 'Boostedt'
    radar['wmo'] = 10132
    radar['lon'] = 10.04687
    radar['lat'] = 54.00438
    radar['alt'] = 124.56
    radars['BOO'] = radar

    radar = {}
    radar['name'] = 'Dresden'
    radar['wmo'] = 10488
    radar['lon'] = 13.76865
    radar['lat'] = 51.12465
    radar['alt'] = 263.36
    radars['DRS'] = radar

    radar = {}
    radar['name'] = 'Eisberg'
    radar['wmo'] = 10780
    radar['lon'] = 12.40278
    radar['lat'] = 49.54066
    radar['alt'] = 798.79
    radars['EIS'] = radar

    radar = {}
    radar['name'] = 'Emden'
    radar['wmo'] = 10204
    radar['lon'] = 7.02377
    radar['lat'] = 53.33872
    radar['alt'] = 58
    radars['EMD'] = radar

    radar = {}
    radar['name'] = 'Essen'
    radar['wmo'] = 10410
    radar['lon'] = 6.96712
    radar['lat'] = 51.40563
    radar['alt'] = 185.10
    radars['ESS'] = radar

    radar = {}
    radar['name'] = 'Feldberg'
    radar['wmo'] = 10908
    radar['lon'] = 8.00361
    radar['lat'] = 47.87361
    radar['alt'] = 1516.10
    radars['FBG'] = radar

    radar = {}
    radar['name'] = 'Flechtdorf'
    radar['wmo'] = 10440
    radar['lon'] = 8.802
    radar['lat'] = 51.3112
    radar['alt'] = 627.88
    radars['FLD'] = radar

    radar = {}
    radar['name'] = 'Hannover'
    radar['wmo'] = 10339
    radar['lon'] = 9.69452
    radar['lat'] = 52.46008
    radar['alt'] = 97.66
    radars['HNR'] = radar

    radar = {}
    radar['name'] = 'Neuhaus'
    radar['wmo'] = 10557
    radar['lon'] = 11.13504
    radar['lat'] = 50.50012
    radar['alt'] = 878.04
    radars['NEU'] = radar

    radar = {}
    radar['name'] = 'Neuheilenbach'
    radar['wmo'] = 10605
    radar['lon'] = 6.54853
    radar['lat'] = 50.10965
    radar['alt'] = 585.84
    radars['NHB'] = radar

    radar = {}
    radar['name'] = 'Offenthal'
    radar['wmo'] = 10629
    radar['lon'] = 8.71293
    radar['lat'] = 49.9847
    radar['alt'] = 245.80
    radars['OFT'] = radar

    radar = {}
    radar['name'] = 'Proetzel'
    radar['wmo'] = 10392
    radar['lon'] = 13.85821
    radar['lat'] = 52.64867
    radar['alt'] = 193.92
    radars['PRO'] = radar

    radar = {}
    radar['name'] = 'Memmingen'
    radar['wmo'] = 10950
    radar['lon'] = 10.21924
    radar['lat'] = 48.04214
    radar['alt'] = 724.40
    radars['MEM'] = radar

    radar = {}
    radar['name'] = 'Rostock'
    radar['wmo'] = 10169
    radar['lon'] = 12.05808
    radar['lat'] = 54.17566
    radar['alt'] = 37
    radars['ROS'] = radar

    radar = {}
    radar['name'] = 'Isen'
    radar['wmo'] = 10873
    radar['lon'] = 12.10177
    radar['lat'] = 48.1747
    radar['alt'] = 677.77
    radars['ISN'] = radar

    radar = {}
    radar['name'] = 'Tuerkheim'
    radar['wmo'] = 10832
    radar['lon'] = 9.78278
    radar['lat'] = 48.58528
    radar['alt'] = 767.62
    radars['TUR'] = radar

    radar = {}
    radar['name'] = 'Ummendorf'
    radar['wmo'] = 10356
    radar['lon'] = 11.17609
    radar['lat'] = 52.16009
    radar['alt'] = 183
    radars['UMM'] = radar

    
    return radars

def plot_radar(radar, ax, proj):

    site = (radar['lon'], radar['lat'], radar['alt'] )
    
    r = np.arange(1, 151) * 1000

    # azimuth array 1 degree spacing
    az = np.linspace(0, 360, 361)[0:-1]

    # build polygons for maxrange rangering
    polygons = wrl.georef.spherical_to_polyvert(r, az, 0,
                                                site,
                                                proj=proj)
    polygons = polygons[..., 0:2]
    polygons.shape = (len(az), len(r), 5, 2)
    polygons = polygons[:, -1, :, :]

    x_loc, y_loc = wrl.georef.reproject(site[0], site[1],
                                        projection_source=proj_wgs,
                                        projection_target=proj)


    # create PolyCollections and add to respective axes
    polycoll = mpl.collections.PolyCollection(polygons, closed=True,
                                              edgecolors='k',
                                              facecolors='k',
                                             linewidths=0.1)
    ax.add_collection(polycoll, autolim=True)

    # plot radar location and information text
    #ax.scatter(x_loc, y_loc, s=40, color='red', edgecolors='black')
    #ax.text(x_loc, y_loc, radar['name'], color='k')

def plot_all_cband2(ax):
    for i in get_radar_locations().keys():

        if i =='EMD':
            pass
        else:
            plot_radar(get_radar_locations()[i], ax , proj_stereo)
            
# create radolan projection object
proj_stereo = wrl.georef.create_osr("dwd-radolan")

# create wgs84 projection object
proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)

In [ ]:
np.unique(sr_cfb)

In [ ]:
%%time

cc=0.5
ff=20

def get_miub_cmap():
    import matplotlib.colors as col
    startcolor = 'white'  # a dark olive
    color1 = '#8ec7ff'#'cyan'    # a bright yellow
    color2 = 'dodgerblue'
    color3 = 'lime'
    color4 = 'yellow'
    color5 = 'darkorange'
    color6 = 'red'
    color7 = 'purple'
    #color6 = 'grey'
    endcolor = 'darkmagenta'    # medium dark red
    colors = [startcolor, color1, color2, color3, color4, color5, color6, endcolor]
    return col.LinearSegmentedColormap.from_list('miub1',colors)

dpr = sr_pns.copy() # para2 = GPM 
ry = gr_pns.copy() # para1 = RY
xx = xl.copy()
yy = yl.copy()

#yy[(ry == 0) & (dpr > 0)] = np.nan
#xx[(ry == 0) & (dpr > 0)] = np.nan

#yy[(ry > 0) & (dpr == 0)] = np.nan
#xx[(ry > 0) & (dpr == 0)] = np.nan

#yy[(ry > 0) & (dpr > 0)] = np.nan
#xx[(ry > 0) & (dpr > 0)] = np.nan

#hd =gr_hmin - (sr_cfb - sr_ele) 
#yy[hd<=1000] = np.nan
#xx[hd<=1000] = np.nan

#yy[(sr_cfb-sr_ele)<=2000] = np.nan
#xx[(sr_cfb-sr_ele)<=2000] = np.nan

#yy[sr_bbh < gr_hmin] = np.nan
#xx[sr_bbh < gr_hmin] = np.nan

yy[sr_sth < 5000] = np.nan
xx[sr_sth < 5000] = np.nan

#yy[sr_cfb<=2000] = np.nan
#xx[sr_cfb<=2000] = np.nan

#yy[ry<=0] = np.nan
#xx[ry<=0] = np.nan
#yy[dpr<=0] = np.nan
#xx[dpr<=0] = np.nan
#yy[sr_bbh<=2] = np.nan
#xx[sr_bbh<=2] = np.nan
#yy[typ!=1] = np.nan
#xx[typ!=1] = np.nan
#yy[(ry <= 0) & (dpr <= 0)] = np.nan
#xx[(ry <= 0) & (dpr <= 0)] = np.nan

mask = ~np.isnan(yy) & ~np.isnan(xx)

mask = ~np.isnan(yy) & ~np.isnan(xx)
fig = pl.figure(figsize=(12,12))
#plt.subplots_adjust(left=0.1, bottom=0.1, right=0.8, top=0.8, wspace=0.2, hspace=0.2)

ax1 = fig.add_subplot(111, aspect='equal')
pl.hist2d(xx[mask],yy[mask], bins=100, cmap=get_miub_cmap())
cb = pl.colorbar(shrink=cc)
cb.set_label("#", fontsize=ff)
cb.ax.tick_params(labelsize=ff)
pl.xlabel('x (km)',fontsize=ff)
pl.ylabel('y (km)',fontsize=ff)
#pl.title('Number of Overpasses 2014-2017 \n DPR rain:yes - RADOLAN rain:yes',fontsize=ff)

pl.title('Number of Overpasses 2014-2018 \n CFB SR >= 1km',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)
plot_all_cband2(ax1)
pl.xlim(-420, 377);
pl.ylim(-4660, -3759)
pl.show()

In [ ]:
%%time
ff=20; cc=0.5
dpr = sr_pns.copy() # para2 = GPM 
ry = gr_pns.copy() # para1 = RY
xx = xl.copy()
yy = yl.copy()

#yy[(ry == 0) & (dpr > 0)] = np.nan
#xx[(ry == 0) & (dpr > 0)] = np.nan

#yy[(ry <= 0) & (dpr <= 0)] = np.nan
#xx[(ry <= 0) & (dpr <= 0)] = np.nan

mask = ~np.isnan(yy) & ~np.isnan(xx)
pl.figure(figsize=(10,10))
pl.hist2d(xx[mask],yy[mask], bins=200)
cb = pl.colorbar(shrink=cc)
cb.set_label("#", fontsize=ff)
cb.ax.tick_params(labelsize=ff)
pl.xlabel('x (km)',fontsize=ff)
pl.ylabel('y (km)',fontsize=ff)
pl.title('Number of Overpasses 2014-2017',fontsize=ff)
pl.xticks(fontsize=ff)
pl.yticks(fontsize=ff)

# Richtige einteilung wäre 
----------------------------------------------
##### Aufgrund er Regenraten berechnung des GPM

$$R = \epsilon ^r \cdot p D_m ^q$$

$\epsilon -> k-Ze$


$r, q, p -> raintyp$


$Dm -> phase$

Tabelleneinteilung 
raintyp ::: -  stratifrom S convective C


phase ::: -  Liquid L und Solid S


rainintensity ::: -  light l moderate m heavy h

> l ::: 0.5 mm/h <= rr < 1 mm/h 

> m ::: 1 mm/h <= rr < 10 mm/h

> h ::: rr > 10 mm/h

    = 1    stratiform
    = 2    convective
    = 3    other


0 - solid
1 - mixed phase
2 - liquid
255 - Missing

In [ ]:
srp, grp = sr_pns.copy(), gr_pns.copy()

SR_SIL = srp[(typ==1.) & (pha==0) & (grp>=0.5) & (grp<1.0)]
SR_SIM = srp[(typ==1.) & (pha==0) & (grp>=1.0) & (grp<10.)]
SR_SIH = srp[(typ==1.) & (pha==0) & (grp>10.)]

SR_SLL = srp[(typ==1.) & (pha==2) & (grp>=0.5) & (grp<1.0)]
SR_SLM = srp[(typ==1.) & (pha==2) & (grp>=1.0) & (grp<10.)]
SR_SLH = srp[(typ==1.) & (pha==2) & (grp>10.)]

SR_CIL = srp[(typ==2.) & (pha==0) & (grp>=0.5) & (grp<1.0)]
SR_CIM = srp[(typ==2.) & (pha==0) & (grp>=1.0) & (grp<10.)]
SR_CIH = srp[(typ==2.) & (pha==0) & (grp>10.)]

SR_CLL = srp[(typ==2.) & (pha==2) & (grp>=0.5) & (grp<1.0)]
SR_CLM = srp[(typ==2.) & (pha==2) & (grp>=1.0) & (grp<10.)]
SR_CLH = srp[(typ==2.) & (pha==2) & (grp>10.)]


GR_SIL = grp[(typ==1.) & (pha==0) & (grp>=0.5) & (grp<1.0)]
GR_SIM = grp[(typ==1.) & (pha==0) & (grp>=1.0) & (grp<10.)]
GR_SIH = grp[(typ==1.) & (pha==0) & (grp>10.)]

GR_SLL = grp[(typ==1.) & (pha==2) & (grp>=0.5) & (grp<1.0)]
GR_SLM = grp[(typ==1.) & (pha==2) & (grp>=1.0) & (grp<10.)]
GR_SLH = grp[(typ==1.) & (pha==2) & (grp>10.)]

GR_CIL = grp[(typ==2.) & (pha==0) & (grp>=0.5) & (grp<1.0)]
GR_CIM = grp[(typ==2.) & (pha==0) & (grp>=1.0) & (grp<10.)]
GR_CIH = grp[(typ==2.) & (pha==0) & (grp>10.)]

GR_CLL = grp[(typ==2.) & (pha==2) & (grp>=0.5) & (grp<1.0)]
GR_CLM = grp[(typ==2.) & (pha==2) & (grp>=1.0) & (grp<10.)]
GR_CLH = grp[(typ==2.) & (pha==2) & (grp>10.)]





In [ ]:
srh = sr_cfb-sr_ele
grh = gr_hmin 

hdiff = abs(grh - srh)

srp, grp = sr_pns.copy(), gr_pns.copy()

SR_SIL = srp[(typ==1.) & (pha==0) & (grp<1.0)]
SR_SIM = srp[(typ==1.) & (pha==0) & (grp>=1.0) & (grp<10.)]
SR_SIH = srp[(typ==1.) & (pha==0) & (grp>10.)]

SR_SLL = srp[(typ==1.) & (pha==2) &  (grp<1.0)]
SR_SLM = srp[(typ==1.) & (pha==2) & (grp>=1.0) & (grp<10.)]
SR_SLH = srp[(typ==1.) & (pha==2) & (grp>10.)]

SR_CIL = srp[(typ==2.) & (pha==0) & (grp<1.0)]
SR_CIM = srp[(typ==2.) & (pha==0) & (grp>=1.0) & (grp<10.)]
SR_CIH = srp[(typ==2.) & (pha==0) & (grp>10.)]

SR_CLL = srp[(typ==2.) & (pha==2) & (grp<1.0)]
SR_CLM = srp[(typ==2.) & (pha==2) & (grp>=1.0) & (grp<10.)]
SR_CLH = srp[(typ==2.) & (pha==2) & (grp>10.)]


GR_SIL = grp[(typ==1.) & (pha==0) & (grp<1.0)]
GR_SIM = grp[(typ==1.) & (pha==0) & (grp>=1.0) & (grp<10.)]
GR_SIH = grp[(typ==1.) & (pha==0) & (grp>10.)]

GR_SLL = grp[(typ==1.) & (pha==2) & (grp<1.0)]
GR_SLM = grp[(typ==1.) & (pha==2) & (grp>=1.0) & (grp<10.)]
GR_SLH = grp[(typ==1.) & (pha==2) & (grp>10.)]

GR_CIL = grp[(typ==2.) & (pha==0) & (grp<1.0)]
GR_CIM = grp[(typ==2.) & (pha==0) & (grp>=1.0) & (grp<10.)]
GR_CIH = grp[(typ==2.) & (pha==0) & (grp>10.)]

GR_CLL = grp[(typ==2.) & (pha==2) & (grp<1.0)]
GR_CLM = grp[(typ==2.) & (pha==2) & (grp>=1.0) & (grp<10.)]
GR_CLH = grp[(typ==2.) & (pha==2) & (grp>10.)]





In [ ]:
srh = sr_cfb-sr_ele
grh = gr_hmin 

hdiff = abs(grh - srh)

srp, grp = sr_pns.copy(), gr_pns.copy()

SR_SIL = srp[(typ==1.) & (pha==0) & (grp<1.0)& (hdiff<=100)]
SR_SIM = srp[(typ==1.) & (pha==0) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
SR_SIH = srp[(typ==1.) & (pha==0) & (grp>10.)& (hdiff<=100)]

SR_SLL = srp[(typ==1.) & (pha==2) &  (grp<1.0)& (hdiff<=100)]
SR_SLM = srp[(typ==1.) & (pha==2) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
SR_SLH = srp[(typ==1.) & (pha==2) & (grp>10.)& (hdiff<=100)]

SR_CIL = srp[(typ==2.) & (pha==0) & (grp<1.0)& (hdiff<=100)]
SR_CIM = srp[(typ==2.) & (pha==0) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
SR_CIH = srp[(typ==2.) & (pha==0) & (grp>10.)& (hdiff<=100)]

SR_CLL = srp[(typ==2.) & (pha==2) & (grp<1.0)& (hdiff<=100)]
SR_CLM = srp[(typ==2.) & (pha==2) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
SR_CLH = srp[(typ==2.) & (pha==2) & (grp>10.)& (hdiff<=100)]


GR_SIL = grp[(typ==1.) & (pha==0) & (grp<1.0)& (hdiff<=100)]
GR_SIM = grp[(typ==1.) & (pha==0) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
GR_SIH = grp[(typ==1.) & (pha==0) & (grp>10.)& (hdiff<=100)]

GR_SLL = grp[(typ==1.) & (pha==2) & (grp<1.0)& (hdiff<=100)]
GR_SLM = grp[(typ==1.) & (pha==2) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
GR_SLH = grp[(typ==1.) & (pha==2) & (grp>10.)& (hdiff<=100)]

GR_CIL = grp[(typ==2.) & (pha==0) & (grp<1.0)& (hdiff<=100)]
GR_CIM = grp[(typ==2.) & (pha==0) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
GR_CIH = grp[(typ==2.) & (pha==0) & (grp>10.)& (hdiff<=100)]

GR_CLL = grp[(typ==2.) & (pha==2) & (grp<1.0)& (hdiff<=100)]
GR_CLM = grp[(typ==2.) & (pha==2) & (grp>=1.0) & (grp<10.)& (hdiff<=100)]
GR_CLH = grp[(typ==2.) & (pha==2) & (grp>10.)& (hdiff<=100)]





In [ ]:
print (SR_SLL.shape ,SR_SLM.shape , SR_SLH.shape )
print (GR_CIL.shape ,GR_CIM.shape , GR_CIH.shape )
print (GR_CLL.shape ,GR_CLM.shape , GR_CLH.shape )


In [ ]:
sr_cfb, gr_hmin

In [ ]:
ff=20
pl.figure(figsize=(25,25))
ax1 = pl.subplot(4,3,1, aspect='equal')
plotplot(ax1,GR_SLL ,SR_SLL, 'SLL')


ax1 = pl.subplot(4,3,2, aspect='equal')
plotplot(ax1,GR_SLM ,SR_SLM, 'SLM')

ax1 = pl.subplot(4,3,3, aspect='equal')
plotplot(ax1,GR_SLH ,SR_SLH, 'SLH')

ax1 = pl.subplot(4,3,4, aspect='equal')
plotplot(ax1,GR_SIL ,SR_SIL, 'SIL')


ax1 = pl.subplot(4,3,5, aspect='equal')
plotplot(ax1,GR_SIM ,SR_SIM, 'SIM')

ax1 = pl.subplot(4,3,6, aspect='equal')
plotplot(ax1,GR_SIH ,SR_SIH, 'SIH')

ax1 = pl.subplot(4,3,7, aspect='equal')
plotplot(ax1,GR_CLL ,SR_CLL, 'CLL')


ax1 = pl.subplot(4,3,8, aspect='equal')
plotplot(ax1,GR_CLM ,SR_CLM, 'CLM')

ax1 = pl.subplot(4,3,9, aspect='equal')
plotplot(ax1,GR_CLH ,SR_CLH, 'CLH')

ax1 = pl.subplot(4,3,10, aspect='equal')
plotplot(ax1,GR_CIL ,SR_CIL, 'CIL')


ax1 = pl.subplot(4,3,11, aspect='equal')
plotplot(ax1,GR_CIM ,SR_CIM, 'CIM')

ax1 = pl.subplot(4,3,12, aspect='equal')
plotplot(ax1,GR_CIH ,SR_CIH, 'CIH')
pl.tight_layout()
pl.savefig("/automount/ftp/velibor/test2.png")
pl.show()

In [ ]:
pd.DataFrame.from_dict(skill_score(GR_SLM, SR_SLM,0.1), orient='index')

In [ ]:
GR_SLM.shape

# Times
----------------------

In [ ]:
import pandas as pd


In [ ]:
sr_time = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_times.npy')
sr_pns = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/sr_pns.npy')
gr_pns = np.load('/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS_compress/gr_pp_ipoli.npy')

In [ ]:
sr_time.shape

In [ ]:
df = pd.DataFrame(dict(sr_pns=sr_pns,
                       gr_pns=gr_pns,
                       sr_gr = sr_pns-gr_pns,time=sr_time))

formater = '%Y-%m-%d %H:%M:%S'
df['Datetime'] = pd.to_datetime(df['time'], format=formater)
df = df.set_index(pd.DatetimeIndex(df['Datetime']))
df = df.drop(['Datetime','time'], axis=1)

In [ ]:
#a = df['sr_gr'].groupby([df.index, pd.TimeGrouper(freq='M')])

In [ ]:
df['sr_gr'].plot()

In [ ]:
plt.show()

In [ ]:
df['time'] = pd.to_datetime(df['time'])  



In [ ]:
df = df.set_index('time')

In [ ]:
df

In [ ]:
plt.subplot(3,1,1)
df['gr_pns'][0:10000].plot()
plt.subplot(3,1,2)

df['sr_pns'][0:10000].plot()

plt.subplot(3,1,3)

df['sr_gr'][0:10000].plot()
plt.show()

In [ ]:
df['sr_gr'].groupby(df.index).mean()

In [ ]:
df['sr_gr'].groupby([df.index, pd.TimeGrouper(freq='Y')])

In [ ]:
df.index

In [ ]:
plt.semilogy(sr_time, sr_pns - gr_pns)
plt.show()

In [ ]:
x

In [ ]:
time

In [ ]:
type(time)

In [ ]:
import datetime
time = np.array(sr_time,dtype='datetime64')


In [ ]:
time


In [ ]:
plt.plot(time, sr_pns - gr_pns)
#pl.ylim(1e-6,1e6)
plt.grid()
plt.axhline(0,linewidth=4, color='r' )
plt.show()